# Relabel IDs

In [ ]:
import logging
import os
import shutil

import pandas as pd
from tqdm import tqdm

In [ ]:
username = ""
source = ""
media = ""

In [ ]:
processed_dir = "../../data/processed_data"
media_to_uid = pd.read_csv(os.path.join(processed_dir, f"{media}_to_uid.csv"))
media_to_uid = media_to_uid.set_index(f"{media}id")["uid"].to_dict()
username_to_uid = {f"{source}@{username}": 0}

In [ ]:
def process(file):
    with open(file, "r") as in_file, open(file + "~", "w") as out_file:
        header = False
        for line in tqdm(in_file):
            if not header:
                header = True
                out_file.write(line)
                continue
            fields = line.strip().split(",")
            item = int(fields[1])
            user = fields[0]
            if item not in media_to_uid:
                logging.warning(
                    f"Skipping item {item} because it was not found in the database. "
                    "This can happen if a new item was released after the database "
                    "was collected. If this happens too often, consider refreshing "
                    "the database."
                )
                continue
            fields[0] = str(username_to_uid[user])
            fields[1] = str(media_to_uid[item])
            out_file.write(",".join(fields) + "\n")
    os.rename(file + "~", file)

In [ ]:
if source != "Training":
    data_path = os.path.join("../../data/recommendations", username)
    src = os.path.join(data_path, f"user_{media}_list.csv")
    process(src)